In [ ]:
import os
import json
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục dữ liệu trên Kaggle
dataset_path = "/kaggle/input/btxrd-data/BTXRD/"
json_dir = os.path.join(dataset_path, "Annotations")
image_dir = os.path.join(dataset_path, "images")

# Lấy danh sách tất cả file JSON và sắp xếp theo thứ tự
json_files = sorted([f for f in os.listdir(json_dir) if f.endswith(".json")])

# Danh sách lưu kết quả
results = []

# Duyệt qua từng file JSON theo thứ tự
for json_file in json_files:
    json_path = os.path.join(json_dir, json_file)

    # Đọc dữ liệu từ JSON
    with open(json_path, "r") as file:
        data = json.load(file)

    if "imagePath" in data and "shapes" in data:
        image_path = os.path.join(image_dir, data["imagePath"])

        # Kiểm tra nếu ảnh tồn tại
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            height, width, _ = image.shape
            total_pixels = height * width  # Diện tích toàn bộ ảnh

            # Tính diện tích vùng ung thư
            tumor_area = 0
            for shape in data["shapes"]:
                label = shape.get("label", "").lower()
                points = np.array(shape.get("points", []), dtype=np.int32)

                # Nếu vùng có đủ điểm để tạo thành đa giác, tính diện tích
                if len(points) > 2:
                    tumor_area += cv2.contourArea(points)

            # Tính tỷ lệ vùng ung thư (%)
            tumor_ratio = (tumor_area / total_pixels) * 100 if total_pixels > 0 else 0

            # Lưu kết quả theo thứ tự
            results.append({
                "Image": data["imagePath"],
                "Total Pixels": total_pixels,
                "Tumor Area": tumor_area,
                "Tumor Ratio (%)": round(tumor_ratio, 2)
            })

# Chuyển kết quả thành DataFrame
df = pd.DataFrame(results)

# Xuất ra file CSV trong thư mục Kaggle output
output_path = "/kaggle/working/tumor_area_ratio.csv"
df.to_csv(output_path, index=False)

# Hiển thị đường dẫn đến file
print(f"📌 Kết quả đã được lưu vào: {output_path}")
print(f"📥 Bạn có thể tải file tại: [Bấm vào đây](./tumor_area_ratio.csv)")

# Hiển thị bảng dữ liệu trên notebook
from IPython.display import display
display(df)

# Vẽ biểu đồ so sánh tỷ lệ vùng ung thư giữa các ảnh theo thứ tự
plt.figure(figsize=(12, 5))
plt.bar(df["Image"], df["Tumor Ratio (%)"], color="red")
plt.xlabel("Tên Ảnh")
plt.ylabel("Tỷ lệ vùng ung thư (%)")
plt.title("Tỷ lệ vùng ung thư trên từng ảnh")
plt.xticks(rotation=90)
plt.show()
